## Importing sql functions

In [0]:
from pyspark.sql.functions import *

In [0]:
path = "/FileStore/tables/yellow_tripdata_2018_01-1.parquet"
data = spark.read.parquet(path)

In [0]:
display(data)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null
1,2018-01-01T00:29:29.000+0000,2018-01-01T00:32:48.000+0000,3,0.5,1,N,143,143,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null
1,2018-01-01T00:38:08.000+0000,2018-01-01T00:48:24.000+0000,2,1.7,1,N,50,239,1,9.0,0.5,0.5,2.05,0.0,0.3,12.35,null,null
1,2018-01-01T00:49:29.000+0000,2018-01-01T00:51:53.000+0000,1,0.7,1,N,239,238,1,4.0,0.5,0.5,1.0,0.0,0.3,6.3,null,null
1,2018-01-01T00:56:38.000+0000,2018-01-01T01:01:05.000+0000,1,1.0,1,N,238,24,1,5.5,0.5,0.5,1.7,0.0,0.3,8.5,null,null
1,2018-01-01T00:17:04.000+0000,2018-01-01T00:22:24.000+0000,1,0.7,1,N,170,170,2,5.5,0.5,0.5,0.0,0.0,0.3,6.8,null,null


## Query 1. 
Add a column named as "Revenue" into dataframe which is the sum of the below columns 'Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount'

In [0]:
display(data.withColumn("Revenue",
              col('Fare_amount')+ col('Extra')+col('MTA_tax')+col('Improvement_surcharge')+col('Tip_amount')+col('Tolls_amount')+col('Total_amount')).display())

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Revenue
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null,11.6
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null,30.6
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null,16.6
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null,69.6
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null,33.1
1,2018-01-01T00:29:29.000+0000,2018-01-01T00:32:48.000+0000,3,0.5,1,N,143,143,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null,11.6
1,2018-01-01T00:38:08.000+0000,2018-01-01T00:48:24.000+0000,2,1.7,1,N,50,239,1,9.0,0.5,0.5,2.05,0.0,0.3,12.35,null,null,24.700000000000003
1,2018-01-01T00:49:29.000+0000,2018-01-01T00:51:53.000+0000,1,0.7,1,N,239,238,1,4.0,0.5,0.5,1.0,0.0,0.3,6.3,null,null,12.6
1,2018-01-01T00:56:38.000+0000,2018-01-01T01:01:05.000+0000,1,1.0,1,N,238,24,1,5.5,0.5,0.5,1.7,0.0,0.3,8.5,null,null,17.0
1,2018-01-01T00:17:04.000+0000,2018-01-01T00:22:24.000+0000,1,0.7,1,N,170,170,2,5.5,0.5,0.5,0.0,0.0,0.3,6.8,null,null,13.6


## Query 2.
Increasing count of total passengers in New York City by area

In [0]:
display(data.groupBy('PULocationID', 'DOLocationID').sum('passenger_count').limit(10))

PULocationID,DOLocationID,sum(passenger_count)
79,116,580
246,249,5073
234,144,7047
161,193,100
231,261,5812
143,211,260
90,231,6447
142,144,971
90,142,2835
249,225,400


## Query 3
Realtime Average fare/total earning amount earned by 2 vendors

In [0]:
display(data.groupBy('VendorID').avg('total_amount'))

VendorID,avg(total_amount)
1,15.127384289902137
2,15.775723474073514


## Query 4
Moving Count of payments made by each payment mode

In [0]:
display(data.groupBy('payment_type').count())

payment_type,count
1,6106416
3,43204
2,2599215
4,11852


## Query 5
Highest two gaining vendor's on a particular date with no of passenger and total distance by cab

In [0]:
display(data.select('VendorID', 'tpep_pickup_datetime', 'passenger_count', 'trip_distance').sort(data.trip_distance.desc()).limit(2))

VendorID,tpep_pickup_datetime,passenger_count,trip_distance
2,2018-01-30T11:41:02.000+0000,1,189483.84
1,2018-01-08T19:44:54.000+0000,0,830.8


## Query 6
Most no of passenger between a route of two location.

In [0]:
display(data.groupBy('PULocationID','DOLocationID').agg(sum('passenger_count').alias('no_of_passenger')).orderBy(col('no_of_passenger').desc()))

PULocationID,DOLocationID,no_of_passenger
264,264,186705
237,236,86737
236,236,78691
236,237,71752
237,237,64499
239,238,44479
239,142,42383
142,239,40365
238,239,37758
141,236,36201


## Query 7
Get top pickup locations with most passengers in last 5/10 seconds.

In [0]:
qd = (data.groupBy(window('tpep_pickup_datetime', '5 seconds'))
      .agg(sum('passenger_count').alias('sum_passenger'), count('PULocationID').alias('top_pick'))
     )
qf = qd.orderBy(F.col('top_pick').desc())
display(qf)

window,sum_passenger,top_pick
"List(2018-01-12T15:15:05.000+0000, 2018-01-12T15:15:10.000+0000)",84,64
"List(2018-01-27T15:15:25.000+0000, 2018-01-27T15:15:30.000+0000)",74,64
"List(2018-01-06T15:15:00.000+0000, 2018-01-06T15:15:05.000+0000)",71,56
"List(2018-01-07T15:15:00.000+0000, 2018-01-07T15:15:05.000+0000)",78,54
"List(2018-01-13T15:15:00.000+0000, 2018-01-13T15:15:05.000+0000)",74,53
"List(2018-01-25T21:40:30.000+0000, 2018-01-25T21:40:35.000+0000)",82,53
"List(2018-01-08T15:15:00.000+0000, 2018-01-08T15:15:05.000+0000)",72,53
"List(2018-01-24T21:35:55.000+0000, 2018-01-24T21:36:00.000+0000)",89,52
"List(2018-01-11T15:15:20.000+0000, 2018-01-11T15:15:25.000+0000)",97,52
"List(2018-01-18T19:05:40.000+0000, 2018-01-18T19:05:45.000+0000)",94,51
